In [1]:
import pandas as pd
from pandas import json_normalize
import mysql.connector

In [2]:
# Sample JSON (replace with actual data)
data = {
    "orderID": 123456,
    "customer": {
        "customerID": "C78910",
        "name": "John Doe",
        "contact": {
            "email": "johndoe@example.com",
            "phone": "+1234567890"
        }
    },
    "shippingAddress": {
        "street": "123 Elm St.",
        "city": "Somewhere",
        "state": "CA",
        "postalCode": "90210",
        "country": "USA"
    },
    "items": [
        {
            "itemID": "A001",
            "description": "Bluetooth Speaker",
            "quantity": 1,
            "price": 99.99
        },
        {
            "itemID": "A002",
            "description": "Wireless Mouse",
            "quantity": 2,
            "price": 25.50
        }
    ],
    "payment": {
        "method": "Credit Card",
        "transactionID": "XYZ12345"
    },
    "status": "Shipped",
    "shipDate": "2024-04-24",
    "deliveryEstimate": "2024-04-29",
    "history": [
        {"date": "2024-04-21", "event": "Order placed"},
        {"date": "2024-04-22", "event": "Payment confirmed"},
        {
            "date": "2024-04-23",
            "event": "Order shipped",
            "details": {
                "courier": "UPS",
                "trackingNumber": "1Z9999AA10123456784"
            }
        }
    ]
}

# Normalize main part of JSON
df_main = json_normalize(data,sep='_')




In [3]:
df_main

,orderID,items,status,shipDate,deliveryEstimate,history,customer_customerID,customer_name,customer_contact_email,customer_contact_phone,shippingAddress_street,shippingAddress_city,shippingAddress_state,shippingAddress_postalCode,shippingAddress_country,payment_method,payment_transactionID
0,123456,"[{'itemID': 'A001', 'description': 'Bluetooth ...",Shipped,2024-04-24,2024-04-29,"[{'date': '2024-04-21', 'event': 'Order placed...",C78910,John Doe,johndoe@example.com,+1234567890,123 Elm St.,Somewhere,CA,90210,USA,Credit Card,XYZ12345


In [4]:
df_items = json_normalize(data['items'], sep='_')
df_items["orderID"]=data['orderID']

In [5]:
df_items

,itemID,description,quantity,price,orderID
0,A001,Bluetooth Speaker,1,99.99,123456
1,A002,Wireless Mouse,2,25.50,123456


In [6]:
df_history = json_normalize(data['history'], sep='_')
df_history["orderID"]=data['orderID']

In [7]:
df_history

,date,event,details_courier,details_trackingNumber,orderID
0,2024-04-21,Order placed,NaN,NaN,123456
1,2024-04-22,Payment confirmed,NaN,NaN,123456
2,2024-04-23,Order shipped,UPS,1Z9999AA10123456784,123456


In [8]:
df_main.drop(columns=["items","history"],axis=1,inplace=True)

In [9]:
merged_df = df_main.merge(df_items, on='orderID', how='left').merge(df_history, on='orderID', how='left')


In [10]:
merged_df

,orderID,status,shipDate,deliveryEstimate,customer_customerID,customer_name,customer_contact_email,customer_contact_phone,shippingAddress_street,shippingAddress_city,...,payment_method,payment_transactionID,itemID,description,quantity,price,date,event,details_courier,details_trackingNumber
0,123456,Shipped,2024-04-24,2024-04-29,C78910,John Doe,johndoe@example.com,+1234567890,123 Elm St.,Somewhere,...,Credit Card,XYZ12345,A001,Bluetooth Speaker,1,99.99,2024-04-21,Order placed,NaN,NaN
1,123456,Shipped,2024-04-24,2024-04-29,C78910,John Doe,johndoe@example.com,+1234567890,123 Elm St.,Somewhere,...,Credit Card,XYZ12345,A001,Bluetooth Speaker,1,99.99,2024-04-22,Payment confirmed,NaN,NaN
2,123456,Shipped,2024-04-24,2024-04-29,C78910,John Doe,johndoe@example.com,+1234567890,123 Elm St.,Somewhere,...,Credit Card,XYZ12345,A001,Bluetooth Speaker,1,99.99,2024-04-23,Order shipped,UPS,1Z9999AA10123456784
3,123456,Shipped,2024-04-24,2024-04-29,C78910,John Doe,johndoe@example.com,+1234567890,123 Elm St.,Somewhere,...,Credit Card,XYZ12345,A002,Wireless Mouse,2,25.50,2024-04-21,Order placed,NaN,NaN
4,123456,Shipped,2024-04-24,2024-04-29,C78910,John Doe,johndoe@example.com,+1234567890,123 Elm St.,Somewhere,...,Credit Card,XYZ12345,A002,Wireless Mouse,2,25.50,2024-04-22,Payment confirmed,NaN,NaN
5,123456,Shipped,2024-04-24,2024-04-29,C78910,John Doe,johndoe@example.com,+1234567890,123 Elm St.,Somewhere,...,Credit Card,XYZ12345,A002,Wireless Mouse,2,25.50,2024-04-23,Order shipped,UPS,1Z9999AA10123456784


In [11]:
merged_df.columns

Index(['orderID', 'status', 'shipDate', 'deliveryEstimate',
       'customer_customerID', 'customer_name', 'customer_contact_email',
       'customer_contact_phone', 'shippingAddress_street',
       'shippingAddress_city', 'shippingAddress_state',
       'shippingAddress_postalCode', 'shippingAddress_country',
       'payment_method', 'payment_transactionID', 'itemID', 'description',
       'quantity', 'price', 'date', 'event', 'details_courier',
       'details_trackingNumber'],
      dtype='object')

In [12]:
len(merged_df.columns)

23

In [13]:
merged_df.dropna(inplace=True)

In [14]:
merged_df.reset_index(drop=True)

,orderID,status,shipDate,deliveryEstimate,customer_customerID,customer_name,customer_contact_email,customer_contact_phone,shippingAddress_street,shippingAddress_city,...,payment_method,payment_transactionID,itemID,description,quantity,price,date,event,details_courier,details_trackingNumber
0,123456,Shipped,2024-04-24,2024-04-29,C78910,John Doe,johndoe@example.com,+1234567890,123 Elm St.,Somewhere,...,Credit Card,XYZ12345,A001,Bluetooth Speaker,1,99.99,2024-04-23,Order shipped,UPS,1Z9999AA10123456784
1,123456,Shipped,2024-04-24,2024-04-29,C78910,John Doe,johndoe@example.com,+1234567890,123 Elm St.,Somewhere,...,Credit Card,XYZ12345,A002,Wireless Mouse,2,25.50,2024-04-23,Order shipped,UPS,1Z9999AA10123456784


In [15]:
merged_df.columns

Index(['orderID', 'status', 'shipDate', 'deliveryEstimate',
       'customer_customerID', 'customer_name', 'customer_contact_email',
       'customer_contact_phone', 'shippingAddress_street',
       'shippingAddress_city', 'shippingAddress_state',
       'shippingAddress_postalCode', 'shippingAddress_country',
       'payment_method', 'payment_transactionID', 'itemID', 'description',
       'quantity', 'price', 'date', 'event', 'details_courier',
       'details_trackingNumber'],
      dtype='object')

In [16]:
#sql connection
conn=mysql.connector.connect(host="localhost", user="root", password="kavi",database="customer")
my_cursor = conn.cursor()
my_cursor.execute("CREATE DATABASE IF NOT EXISTS customer")
print("database created successfully")

database created successfully


In [20]:
#customer table creation
my_cursor.execute('''CREATE TABLE IF NOT EXISTS customer_details(
                  orderID INT,
                  status VARCHAR(255),
                  shipDate TIMESTAMP,
                  deliveryEstimate TIMESTAMP,
                  customer_customerID TEXT,
                  customer_name VARCHAR(255),
                  customer_contact_email TEXT,
                  customer_contact_phone BIGINT,
                  shippingAddress_street TEXT,
                  shippingAddress_city VARCHAR(255),
                  shippingAddress_state VARCHAR(255),
                  shippingAddress_postalCode BIGINT ,
                  shippingAddress_country TEXT,
                  payment_method  VARCHAR(255),
                  payment_transactionID TEXT,
                  itemID VARCHAR(255),
                  description TEXT,
                  quantity INT,
                  price FLOAT,
                  date TIMESTAMP,
                  event VARCHAR(255),
                  details_courier TEXT,
                  details_trackingNumber TEXT
                  
                  )''')
print("Table Created successfully")

Table Created successfully


In [21]:
# insert values into customer table
insert_query='''INSERT INTO customer_details(
                  orderID,
                  status,
                  shipDate,
                  deliveryEstimate,
                  customer_customerID,
                  customer_name,
                  customer_contact_email,
                  customer_contact_phone,
                  shippingAddress_street,
                  shippingAddress_city,
                  shippingAddress_state,
                  shippingAddress_postalCode,
                  shippingAddress_country,
                  payment_method,
                  payment_transactionID,
                  itemID,
                  description,
                  quantity,
                  price,
                  date,
                  event,
                  details_courier,
                  details_trackingNumber)
                  VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'''
data=merged_df.values.tolist()
my_cursor.executemany(insert_query,data)
print("Inserted successfully")

Inserted successfully
